In [2]:
from FAdo.reex import *

In [122]:
NT = {}
class Node:
    
    def __init__(self,expr,string):
        self.type = type(expr)
        self.path = string
        self.exp = expr
        if self.type == position:
            self.posix = expr.val[1]
        NT[self.path] = self
    
    def set_left(self,n):
        NT[self.path+'1'] = n
        
    def set_right(self,n):
        NT[self.path+'2'] = n

    def get_left(self):
        return NT.get(self.path+'1')
    
    def get_right(self):
        return NT.get(self.path+'2')
        
    def get_parent(self):
        if self.path=="1":
            return NT[self.path]
        return NT[self.path[:-1]]
    
    def printTree(self):
        if NT.get(self.path):
            print self.exp
            if NT.get(self.path+'1'):
                l = NT[self.path+'1']
                l.printTree()
            if NT.get(self.path+'2'):
                r = NT[self.path+'2']
                r.printTree()
        
    def get_root(self):
        return NT.get("1")
    
    def check_reex(self):
        if self.type == 'atom':
            if self.get_left() or self.get_right():
                return False
        elif self.type == 'star':
            if self.get_left() is None or self.get_right():
                return False
            if self.get_left().check_reex() is False:
                return False
        else:
            if self.get_left() is None or self.get_right() is None:
                return False
            if self.get_left().check_reex() is False or self.get_right().check_reex() is False:
                return False
        return True
    
    def reflexive(self,n):
        if LCA(self,n)==n.path:
            return True
        else:
            return False

    def followList(self):
        f_list = []
        for x in NT:
            node = NT.get(x)
            if self == node:
                continue
            if q_follow(self,node):
                f_list.append(node.exp)
        return f_list
    
    # list of first atoms from a reg exp using lemma 2.3
    def first(self):
        f_list = []
        for x in NT:
            node = NT.get(x)
            if self == node:
                continue
            if isFirst(self,node):
                f_list.append(node.exp)
        return f_list
    
    # list of last atoms from a reg exp using lemma 2.3
    def last(self):
        f_list = []
        for x in NT:
            node = NT.get(x)
            if self == node:
                continue
            if isLast(self,node):
                f_list.append(node.exp)
        return f_list
    

def get_root():
    return NT.get("1")

# Last Common Ancestor
def LCA(p,q):
    str1 = p.path
    str2 = q.path
    ret = ""
    i = 0
    while i < len(str1) and i < len(str2):
        if str1[i]!=str2[i]:
            break
        ret = ret+str1[i]
        i = i+1
    return ret


In [123]:
"""
atom -> val
star -> arg
other -> arg1 /\ arg2
"""
def eTree(reg_exp,path):
    node = Node(reg_exp,path)
    if type(reg_exp)==position:
        return
    elif type(reg_exp)==star:
        son = eTree(node.exp.arg,path+"1")
    else:
        son1 = eTree(node.exp.arg1,path+"1")
        son2 = eTree(node.exp.arg2,path+"2")

In [124]:
#lemma 2.2
def q_follow(p,q):
    node1 = NT.get(LCA(p,q))
    list1 = []
    for x in node1.get_right().exp.first():
        list1.append(x.val)
    list2 = []
    for x in node1.get_left().exp.last():
        list2.append(x.val)
    if type(node1.exp)==concat and str(q.exp) in list1 and str(p.exp) in list2:
        return True
    while type(node1.exp)!=star:
        if node1.path=="1":
            return False
        node1 = node1.get_parent()
    list1 = []
    for x in node1.exp.first():
        list1.append(x.val)
    list2 = []
    for x in node1.exp.last():
        list2.append(x.val)
    if str(q.exp) in list1 and str(p.exp) in list2:
        return True
    return False


In [128]:
a = str2regexp("a(b+cd)*")
a = a.marked()
print a
NT = {}
eTree(a,"1")
p = NT.get("11") # atom(a)
q = NT.get("12") # star(b+cd)
q1 = NT.get("1211") # atom(b)
q2 = NT.get("12121") # atom(c)
q3 = NT.get("12122") # atom(d)
print "atom(a) is followed by atom(b): %s"%q_follow(p,q1)
print "atom(a) is followed by atom(c): %s"%q_follow(p,q2)
print "atom(a) is followed by atom(d): %s"%q_follow(p,q3)
print "atom(a) is followed by:",p.followList()
print "star(b+cd) first:",q.first()
print "star(b+cd) last:",q.last()

('a', 1) (('b', 2) + (('c', 3) ('d', 4)))*
atom(a) is followed by atom(b): False
atom(a) is followed by atom(c): False
atom(a) is followed by atom(d): False
atom(a) is followed by: []
star(b+cd) first: []
star(b+cd) last: []


In [126]:
# boolean that states if an atom is the first from the right child of a concatenation
def SupFirst(n):
    father = n.get_parent()
    if father.type == concat and father.get_right()==n and father.get_left()!=None:
        return True
    else:
        return False

# boolean that states if an atom is the last from the left child of a concatenation
def SupLast(n):
    father = n.get_parent()
    if father.type == concat and father.get_left()==n and father.get_right()!=None:
        return True
    else:
        return False

# pointer to the first(right child) of a concatenation
def pSupFirst(n):
    node = n
    while not SupFirst(node):
        if node.path == "1":
            return None
        node = node.get_parent()
    return node

# pointer to the last(left child) of a concatenation
def pSupLast(n):
    node = n
    while not SupLast(node):
        if node.path == "1":
            return None
        node = node.get_parent()
    return node
        

In [129]:
# lemma 2.3
def isFirst(n,p):
    if pSupFirst(p) == None:
        return False
    if p.type == position and p.reflexive(n) and n.reflexive(pSupFirst(p)):
        return True
    return False

def isLast(n,p):
    if pSupLast(p) == None:
        return False
    if p.type == position and p.reflexive(n) and n.reflexive(pSupLast(p)):
        return True
    return False

In [138]:
P = str2regexp("c*(a+b)*a*c")
NT = {}
eTree(P.marked(),"1")
q = NT.get("112")
get_root().printTree()
print "first:",q.first()
print "last:",q.last()

((('c', 1)* (('a', 2) + ('b', 3))*) ('a', 4)*) ('c', 5)
(('c', 1)* (('a', 2) + ('b', 3))*) ('a', 4)*
('c', 1)* (('a', 2) + ('b', 3))*
('c', 1)*
('c', 1)
(('a', 2) + ('b', 3))*
('a', 2) + ('b', 3)
('a', 2)
('b', 3)
('a', 4)*
('a', 4)
('c', 5)
first: [position('a', 4)]
last: [position('a', 4)]


In [132]:
# follow defined by lemma 2.5 (q follows p)
def follow(p,q):
    node1 = pSupFirst(q)
    node2 = pSupLast(p)
    if node1.path == "1" or node2.path == "1" or p.type != position or q.type != position:
        return False
    if p.reflexive(node1.get_parent()) and q.reflexive(node2.get_parent()):
        return True
    return False
    

In [133]:
# epsilon acceptance by lemma 2.6
def ewp_SupFirst(p,q):
    node = pSupFirst(q)
    if node.path == "1" or p.type != position or q.type != position or not follow(p,q):
        return False
    node = node.get_parent()
    if node.reflexive(pSupLast(p)):
        return True
    return False

In [41]:
# word acceptance by a regexp by lemma 2.7


In [136]:
NT = {}
kkk = str2regexp("c*(a+b)*a*c")
kkk = kkk.marked()
eTree(kkk,"1")
print NT.get("11").exp

(('c', 1)* (('a', 2) + ('b', 3))*) ('a', 4)*
